## 서울시 자전거 따릉이 데이터 전처리
* 데이터 출처 : [데이터셋> 데이터 이용하기 | 서울열린데이터광장](http://data.seoul.go.kr/dataList/datasetView.do?infId=OA-15182&srvType=F&serviceKind=1&currentPageNo=1)
* 전체 데이터를 가져와서 연도별로 병합(concat)을 해주면 1G가 넘는 용량으로 파일이 만들어 집니다.
* 1G가 넘는 파일을 불러와서 작업을 하려면 장비에 부담이 되기 때문에 데이터를 줄여줍니다.

In [ ]:
import pandas as pd

In [ ]:
%ls data/seoul-bike-*

In [ ]:
year = 2019

In [ ]:
%time df = pd.read_csv(f"data/seoul-bike-{year}.csv", low_memory=False)
df.shape

In [ ]:
df.columns

In [ ]:
%time df["자전거번호SPB"] = df["자전거번호"].apply(lambda x : x.split("-")[1])

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# 데이터가 너무 커서 로드하는데 시간이 오래 걸리거나 전처리에 시간이 오래 걸린다.
# 자원을 효율적으로 사용하기 위해 용량이 큰 텍스트 데이터를 빼주고 나중에 join이나 merge를 통해 불러오도록 한다.
df_reduce = df[['자전거번호SPB', '대여일시', '대여대여소번호', '반납일시', '반납대여소번호',
       '이용시간(분)', '이용거리(M)']]
df_reduce.shape

In [ ]:
df_reduce.head()

In [ ]:
%time df_reduce.to_csv(f"data/seoul-bike-{year}-reduce.csv", index=False)

In [ ]:
%time df_reduce_file = pd.read_csv(f"data/seoul-bike-{year}-reduce.csv", low_memory=False)
df_reduce_file.shape

In [ ]:
# 용량이 절반정도 줄어 들었다.
%ls -lah data/seoul-bike-*